In [1]:
import os

import torch
from diffusers import StableDiffusionPipeline, DDPMScheduler
from map_generation.osm_dataset import TextToImageDataset
from config import DEVICE, DATA_DIR
from torch.utils.data import DataLoader
import map_generation.diff_hugging as diff_hugging
import config

In [2]:

diff_hugging.main(config)

getting data


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2144 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/marcin/.cache/huggingface/datasets/mprzymus___parquet/mprzymus--osm_tiles_small-199229e0bb266c92/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                 | Params
------------------------------------------------------
0 | text_encoder | CLIPTextModel        | 123 M 
1 | vae          | AutoencoderKL        | 83.7 M
2 | unet         | UNet2DConditionModel | 579 M 
------------------------------------------------------
579 M     Trainable params
206 M     Non-trainable params
786 M     Total params
3,144.397 Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/home/marcin/Documents/mgr/projekt_n_w/own/.venv/lib/python3.10/site-packages/diffusers/configuration_utils.py:135: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
Epoch 0, global step 2144: 'train/loss' reached 0.13991 (best 0.13991), saving model to '/home/marcin/Documents/mgr/projekt_n_w/own/workflow_raster/model/testing_workflow.ckpt' as top 1
Epoch 1, global step 4288: 'train/loss' reached 0.12888 (best 0.12888), saving model to '/home/marcin/Documents/mgr/projekt_n_w/own/workflow_raster/model/testing_workflow.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=2` reached.


saving...


The config attributes {'predict_epsilon': True} were passed to DDPMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
The config attributes {'predict_epsilon': True} were passed to DPMSolverMultistepScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


# Testing if loss is reasonable

To make sure saving works.

In [2]:
from tqdm import tqdm
import torch.nn.functional as F

def evaluate(dl, pipeline, DEVICE=DEVICE):
    total_loss = 0.0
    for img, caption in tqdm(dl):
        with torch.no_grad():
            latents = pipeline.vae.encode(
                            img.to(DEVICE)
                        ).latent_dist.sample()
            latents = latents * pipeline.vae.config.scaling_factor

            # Sample noise that we'll add to the latents
            noise = torch.randn_like(latents)

            bsz = latents.shape[0]
            # Sample a random timestep for each image
            timesteps = torch.randint(
                0,
                noise_scheduler.config.num_train_timesteps,
                (bsz,),
                device=latents.device,
            )
            timesteps = timesteps.long()

            # Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Get the text embedding for conditioning
            encoder_hidden_states = pipeline.text_encoder(caption.to(DEVICE))[0]

            # Get the target for loss depending on the prediction type
            if noise_scheduler.config.prediction_type == "epsilon":
                target = noise
            elif noise_scheduler.config.prediction_type == "v_prediction":
                target = noise_scheduler.get_velocity(latents, noise, timesteps)
            else:
                raise ValueError(
                    f"Unknown prediction type {noise_scheduler.config.prediction_type}"
                )

            # Predict the noise residual and compute loss
            model_pred = pipeline.unet(
                noisy_latents, timesteps, encoder_hidden_states
            ).sample

        
        loss = F.mse_loss(
            model_pred.float(), target.float(), reduction="mean"
            )
        total_loss += loss.item()
    return total_loss / len(dl)

In [4]:
ds = TextToImageDataset(DATA_DIR)
dl = DataLoader(ds, batch_size=1)
model_name = f"{config.CHECKPOINTS_DIR}/{config.MODEL_NAME}_pipeline"
pipeline = StableDiffusionPipeline.from_pretrained(model_name)
noise_scheduler = DDPMScheduler.from_pretrained(
        model_name, subfolder="scheduler"
    )
evaluate(dl, pipeline.to(DEVICE))

Found cached dataset parquet (/home/marcin/.cache/huggingface/datasets/mprzymus___parquet/mprzymus--osm_tiles_small-199229e0bb266c92/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/marcin/.cache/huggingface/datasets/mprzymus___parquet/mprzymus--osm_tiles_small-199229e0bb266c92/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b7cef2b1443476bf.arrow
The config attributes {'predict_epsilon': True} were passed to DPMSolverMultistepScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
The config attributes {'predict_epsilon': True} were passed to DDPMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
100%|██████████| 2144/2144 [04:13<00:00,  8.45it/s]


0.13314637700256496